<a href="https://colab.research.google.com/github/tuchinhung/Quantu-ML/blob/main/Comparacao_do_modelo_Classico_VS_Quantico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture 

# Comment this out if you don't want to install pennylane from this notebook
!pip install pennylane
# Comment this out if you don't want to install torch from this notebook
!pip install torch
# Comment this out if you don't want to install matplotlib from this notebook
!pip install matplotlib

Modelo Classico regressao linear

In [3]:
import torch
from torch.autograd import Variable
np.random.seed(42)

data = torch.tensor([(0., 0.), (0.1, 0.1), (0.2, 0.2)])
print(data)

tensor([[0.0000, 0.0000],
        [0.1000, 0.1000],
        [0.2000, 0.2000]])


In [4]:
def model(phi, x=None):
  return x*phi

In [5]:
# Função de custo (a-b)^2 Perda de erro quadrático
def loss(a, b):
    return torch.abs(a - b)**2

# Perda média
def average_loss(phi):
    c = 0
    for x, y in data:
        c += loss(model(phi, x=x),  y) # Função de perda de erro quadrático
    return c

phi_new = Variable(torch.tensor(0.1), requires_grad=True)
print(phi_new)
opt = torch.optim.Adam([phi_new], lr=0.02)
print(opt)

tensor(0.1000, requires_grad=True)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.02
    weight_decay: 0
)


In [17]:
for i in range(10):
  loss1 = average_loss(phi_new)
  print(i, loss1.item())
  loss1.backward()
  opt.step()

0 0.5126111421711157
1 0.4632816317036419
2 0.4120567699877299
3 0.36003579372095323
4 0.30834206718844237
5 0.25809614152959726
6 0.21038886312737543
7 0.16625304031458862
8 0.12663759203437513
9 0.0923828137019544


Modelo Quantico Circuito

In [7]:
from pennylane import *
import torch
from torch.autograd import Variable
np.random.seed(42)

data1 = [(0., 0.), (0.1, 0.1), (0.2, 0.2)]
print(data1)

[(0.0, 0.0), (0.1, 0.1), (0.2, 0.2)]


In [8]:
# 'default.qubit': um simulador de estado simples de arquiteturas de circuitos quânticos baseadas em qubit.
# com 2 qubit
dev = device('default.qubit', wires=2)

@qnode(dev, interface='torch')
def circuit_model(phi, x=None):

    templates.AngleEmbedding(features=[x], wires=[0])
    templates.BasicEntanglerLayers(weights=phi, wires=[0, 1])
    return expval(PauliZ(wires=1)) # Valor de expectativa do observável fornecido.
    # o circuito a seguir retorna o valor esperado do operador Pauli Z observável no fio 1


In [9]:
# Função de custo (a-b)^2 Perda de erro quadrático
def loss(a, b):
    return torch.abs(a - b)**2

# Perda média
def average_loss(phi):
    c = 0
    for x, y in data:
        c += loss(circuit_model(phi, x=x),  y) # Função de perda de erro quadrático
    return c

phi_new = Variable(torch.tensor([[0.1, 0.2], [-0.5, 0.1]]), requires_grad=True)
print(phi_new)
opt = torch.optim.Adam([phi_new], lr=0.02)
print(opt)

tensor([[ 0.1000,  0.2000],
        [-0.5000,  0.1000]], requires_grad=True)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.02
    weight_decay: 0
)


In [21]:
for i in range(10):
  loss1 = average_loss(phi_new)
  print(i, loss1.item())
  loss1.backward()
  opt.step()

0 0.4729897527228859
1 0.4283389589112073
2 0.3791347178793804
3 0.326653431373143
4 0.2724367709285855
5 0.21829022715544374
6 0.16625988531537197
7 0.11859690984432532
8 0.07770057601894903
9 0.04604164153859334
